# Tracking useful datasets

1. Start
    1. train_base
    1. census_data
2. Preprocessing
    1. train_base
    1. thin_census_data (converted form of census for join)
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Installation of libraries as required

!pip install sklearn

# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Define custom functions or classes

In [ ]:
# None required right now

# Pre-Processing data

1. Import the dataset
2. Generate high level summaries and pair plots

## Import data

In [ ]:
train_base = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')
test_base = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv')

census_data = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv')
sample_submission = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv')

train_base.head(5)

In [ ]:
census_data.head(5)

In [ ]:
sample_submission.head(5)

#### Notes:

1. Additional census attributes need to be mapped through cfips attribute
2. Census data needs to be transformed into a thin format for usablity and joins
3. Census data is yearly, whereas the microbusiness data is monthly. We will need to keep this in ming when modelling later
4. Final submission needs a key based on cfips and date combination. Thus we will be working with 2 sets of keys to join and model data.
    1. cfips + full date
    2. cfips + year (for joining census data)

## Basic summary
We will try and understand the type of data we will work with

In [ ]:
train_base.describe()

In [ ]:
train_base.isnull().any()

In [ ]:
train_base.dtypes

In [ ]:
census_data.isnull().sum(axis = 0)

## Pre-processing

1. Convert census data into thin format
1. Fill in the missing data in census data
1. Add relevant keys to census and train data
1. Validate if all keys are present in both datasets (helps determine potential upcoming missing values in data)

We may need to perform additional processing based on the validation results

### Transform census data

In [ ]:
census_data_cols = census_data.columns.values

# We will use "_" to seperate each string and create a new thin dataframe
# This dataframe will be at a "cfips" and "year" level to enable join with the base dataset

temp = pd.DataFrame(census_data_cols, index= None)

temp.rename(columns = {0:"col_name"}, inplace = True)

# Remove cfips to make life easy

temp = temp.loc[temp['col_name'] != "cfips",:]

temp['year'] = temp['col_name'].str.slice(-4).astype('int')
temp['temp'] = "_"+temp['year'].astype('str')

#temp['new_col_name'] = temp['col_name'].str.replace(temp['temp'], "")
# Since expected method does not work, we will remove the last 5 characters for ease

temp['new_col_name'] = temp['col_name'].str.slice(start = 0, stop = -5).astype('str')

temp.drop(columns = ("temp"), inplace = True)
temp

#### Convert census to thin format

We will run a loop to iterate through our gegerated columns, and append data to empty dataframe
We will summarize the dataframe to make format consistent without loss of data
Hopefully there is a better way to do this

In [ ]:
new_cols = list(temp['new_col_name'].unique())

# Add the year and cfips columns 
new_cols.insert(0, ("year"))
new_cols.insert(0, ("cfips"))

# Make empty dataframe
thin_census_data = pd.DataFrame(columns = new_cols)

# Generate all possible combinations of cfips and year from just census data

for row in temp.itertuples():
    col_name = row.col_name
    year = row.year
    new_col_name = row.new_col_name
    
    # Make data to append/join
    temp_data = census_data.loc[:,("cfips",col_name)]# Get the required column along with IDs
    temp_data["year"] = year # Add year column
    
    temp_data = temp_data.loc[:,("cfips","year",col_name)] # Reorder columns for consistency
    temp_data.rename(columns = {col_name:new_col_name}, inplace = True)
    
    # Append data to out main dataframe
    thin_census_data = pd.concat((thin_census_data,temp_data), axis = 0)


# Summarize the dataset to get the final 
thin_census_data = thin_census_data.groupby(by = ["cfips","year"], as_index = False).first().reset_index()
thin_census_data.drop(columns = 'index', inplace = True)

thin_census_data.tail()

#### Validate the transformed dataset

We will check if everything went smoothly using simple summaries across original and converted dataset

In [ ]:
# Original summary
census_data.describe()

In [ ]:
thin_census_data.groupby(by = ['year']).min()
thin_census_data.groupby(by = ['year']).max()

### Process the train dataset

We will extract relevant information from the dataset and join the census data to make the final dataset

1. Extract the year and month out of the date columns
2. Reorder columns for ease of reading

In [ ]:
train_base.head()

### Merge train and test datasets

Simplifies processing and appending required data 

In [ ]:
train_base['is_test'] = 0
test_base['is_test'] = 1

all_data = train_base.append(test_base)
all_data.reset_index(drop=True, inplace=True)

all_data.head()

### Add required columns and tweak data

In [ ]:
# Extract the year from dates
all_data['first_day_of_month'] = pd.to_datetime(all_data['first_day_of_month'])

# Sort data to ensure proper imputations of data
all_data.sort_values(by = ['cfips','first_day_of_month'], inplace = True, ignore_index = True)

In [ ]:
# Extracting important numbers from date
all_data['year'] = all_data['first_day_of_month'].dt.year
all_data['month'] = all_data['first_day_of_month'].dt.month


In [ ]:
# Add the state and county names where missing

all_data['county'] = all_data.groupby('cfips')['county'].ffill()
all_data['state'] = all_data.groupby('cfips')['state'].ffill()

all_data.tail()

In [ ]:
# Assign numerical ids to each entry within cfip

# Number of densities for each county, will assign incremental number for each succeeding date
all_data["time_step"] = all_data.groupby(['cfips'])['first_day_of_month'].cumcount()

# Use factorize to assign numerical values. The final [0] is to pick the numbers assigned and ignore the index with text
all_data['county_num'] = (all_data['county'] + all_data['state']).factorize()[0]
# Same as above for states
all_data['state_num'] = all_data['state'].factorize()[0]

all_data.tail()

In [ ]:
lag = 1

# Shift moves a past time step to 1 unit forward. Thus we do bfill since the first date will have blank values
all_data[f'mbd_lag_{lag}'] = all_data.groupby('cfips')['microbusiness_density'].shift(lag).bfill()


all_data[f'mbd_lag_ratio_{lag}'] = (all_data['microbusiness_density'] / all_data[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1


all_data

In [ ]:
lag = 1

# Shift moves a past time step to 1 unit forward. Thus we do bfill since the first date will have blank values
all_data[f'mbd_lag_{lag}'] = all_data.groupby('cfips')['microbusiness_density'].shift(lag).bfill()

########
# Calculate a difference ratio for lag 1 MBD  
all_data[f'mbd_lag_ratio_{lag}'] = (all_data['microbusiness_density'] / all_data[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
# Reassign infinite ratios as 0
all_data.loc[(all_data[f'mbd_lag_{lag}']==0), 'dif'] = 0
# Reassign infinite ratios with recent uptake in MBD as 1.
all_data.loc[(all_data[f'microbusiness_density']>0) & (all_data[f'mbd_lag_{lag}']==0), f'mbd_lag_ratio_{lag}'] = 1

# convert to absolute value (not sure why)
all_data[f'mbd_lag_ratio_{lag}'] = all_data[f'mbd_lag_ratio_{lag}'].abs()

# Plot to check any inconsistencies
all_data.groupby('time_step')[f'mbd_lag_ratio_{lag}'].sum().plot()

### Simple reorder for more sensible data

In [ ]:
# Reorder columns
reorder_cols = ('row_id', 'cfips', 'year', 'month', 'first_day_of_month', 'county', 'state', 'active', 'microbusiness_density')
all_data = all_data.loc[:,reorder_cols]

all_data.head()

## Cross-Validate both datasets

1. We will check if we have match for all (or most) keys in dataset

Observations:
1. Census data has extra counties included
1. Census data is not available for 2022 (which is logical).
    1. THus we may need to generate some projecttions/estimates for 2022. We can either use same 2021 numbers or determine average growth rate based on bucketed counties


In [ ]:
train_cfips = train_base['cfips'].unique()
train_years = train_base['year'].unique()

census_cfips = thin_census_data['cfips'].unique()
census_years = thin_census_data['year'].unique()

#### Check county IDs

In [ ]:
print("Training data")
print(train_cfips)
print("size = ",train_cfips.shape)

print("\nCensus Data")
print(census_cfips)
print("size = ",census_cfips.shape)

# Print missing from each sets
print("Missing from census data (but are present in training data):")
print(train_cfips[~np.in1d(train_cfips, census_cfips)])

print("Missing from training data (and are available in census):")
print(census_cfips[~np.in1d(census_cfips, train_cfips)])

#### Check years

In [ ]:
print("Training data")
print(train_years)
print("size = ",train_years.shape)

print("\nCensus Data")
print(census_years)
print("size = ",census_years.shape)

# Print missing from each sets
print("Missing from census data (but are present in training data):")
print(train_years[~np.in1d(train_years, census_years)])

print("Missing from training data (and are available in census):")
print(census_years[~np.in1d(census_years, train_years)])

# EDA

As we have some relevant information on the data, we can perform EDA to establish a high level understanding before proceeding.

## EDA - Training data

Visualize the timeline using line charts

### Microbusiness density over time

Since we have few counties doing cnonsiderably better than others (again expected information), we may need to bucket counties into groups for easy managable visualizations

In [ ]:
# Making a copy
eda_train_data = train_base

In [ ]:
# Quick pair plot
import seaborn as sns

sns.pairplot(data = eda_train_data, hue='year')

In [ ]:
# Bucket Counties based on micro density variable
# We will have to tag outliers and then decile the data
# Based on the pair plot, we will tag the entries with 
# We will use deciles for initial view
# Keep in mind, decile ranking assigns first label to lowest decile

# Summarize data to get avergae density for each county over years.
temp_avg_density = eda_train_data.groupby(['cfips']).agg({'microbusiness_density':'mean'}).reset_index()

# Store list of cfips based on graphs
# We will treat average above 30 as an outlier
outlier_cfips = temp_avg_density.loc[temp_avg_density['microbusiness_density'] >= 30.0, ['cfips']]
outlier_cfips.loc[:,'microdensity_buckets'] = "outlier"

# We bucket the rest
normal_cfips = temp_avg_density.loc[~temp_avg_density['cfips'].isin(outlier_cfips['cfips'].unique()),['cfips','microbusiness_density']]
normal_cfips.loc[:,['microdensity_buckets']] = pd.qcut(temp_avg_density['microbusiness_density'],10, labels = False)

# normal_cfips.head()
normal_cfips = normal_cfips.loc[:,["cfips","microdensity_buckets"]]

cfip_density_buckets = pd.concat((normal_cfips, outlier_cfips), axis = 0).reset_index()
# cfip_density_buckets


# Plot for inspection
sns.pairplot(data = temp_avg_density)

In [ ]:
# Assign the average micro business density buckets to data
eda_train_data = pd.merge(left = eda_train_data, right = cfip_density_buckets,on = 'cfips', how = 'left')

# Trying to see how data ends up
# Quick observations
# Outlier buckets seem to have counties that started small (but had relatively higher density to start off with), but have high density over years
# These counties may already have decent population
temp = eda_train_data.groupby(('microdensity_buckets'), as_index = False).agg(min_density = ("microbusiness_density",np.min),max_density = ("microbusiness_density",np.max))
temp

In [ ]:
# Through visual inspection, we need to classify certain counties as outliers to get a better idea of trends
# Quick scatter plot to visualise the outliers

g = sns.FacetGrid(data = eda_train_data,col = 'microdensity_buckets', col_wrap = 3)

g.map(sns.lineplot, "first_day_of_month", "microbusiness_density")

### Active over time

We will check the active attribute and see if we can get some information from it.
Also, since we will not have this attribute available for test set, we will need to figure out a imputation for this variable.

Good idea is to check coorelation with other attributes to make the task easy

In [ ]:
temp_avg_active = eda_train_data.groupby(['cfips'], as_index=False).agg({'active':'mean'})

# Store list of cfips based on graphs
# Difficult to see actual values due to extreme outlier. Remove and recheck

# We will do trial and error to evaluate the best cutoff based on average active businesses
# Looking at granular plots, a good cutoff for active attribute outliers at 20k
outlier_cfips_active = temp_avg_active.loc[temp_avg_active['active'] >= 2.0e4, ['cfips']]

outlier_cfips_active['active_buckets'] = "outlier"
# None here. We will merge it regardless for redundancy
zero_cfips_active = temp_avg_active.loc[temp_avg_active['active'] == 0, 'cfips']

if len(zero_cfips_active) != 0:
    zero_cfips_active['active_buckets'] = "zero"
    outlier_cfips_active = pd.concat((outlier_cfips_active, zero_cfips_active), axis = 0)

# Generate buckets for the remaining counties
temp_avg_active = temp_avg_active.loc[~temp_avg_active['cfips'].isin(outlier_cfips_active['cfips']), :]
temp_avg_active['active_buckets'] = pd.qcut(temp_avg_active['active'],5, labels = False)
temp_avg_active = temp_avg_active.loc[:,['cfips','active_buckets']]

# Final list of buckets
active_buckets = pd.concat((outlier_cfips_active, temp_avg_active))
active_buckets = active_buckets.reset_index(drop="index")

active_buckets

In [ ]:
# Assign the average micro business density buckets to data
eda_train_data = pd.merge(left = eda_train_data, right = active_buckets,on = 'cfips', how = 'left')

# Trying to see how data ends up
# Quick observations
# Outlier buckets seem to have counties that started small (but had relatively higher density to start off with), but have high density over years
# These counties may already have decent population

temp = eda_train_data.groupby(('active_buckets'), as_index = False).agg(min_density = ("active",np.min),max_density = ("active",np.max))
temp

In [ ]:
# Quick scatter plot to visualise the outliers
# sns.lineplot(data = temp, x = "first_day_of_month", y = "active", hue = "active_buckets")
temp = eda_train_data.loc[eda_train_data['active_buckets'] != "outlier",:]
g = sns.FacetGrid(data = temp, col = "active_buckets", col_wrap = 3)
g.map(sns.lineplot, "first_day_of_month", 'active')


## 

## Census Data

We will plot census data and see if we can see a simple way to forecast it

The direct pairplot takes too long to process. We will try and shrink the data a bit by grouping it by states


In [ ]:
# Updating the dataset

eda_census_data = census_data.copy()

In [ ]:
eda_census_data.columns

In [ ]:
# The direct pairplot takes too long to process. We will try and shrink the data a bit by grouping it by states

col_set_2017 = ['cfips', 'pct_bb_2017', 'pct_college_2017', 'pct_foreign_born_2017', 'pct_it_workers_2017', 'median_hh_inc_2017']
col_set_2018 = ['cfips', 'pct_bb_2018', 'pct_college_2018', 'pct_foreign_born_2018', 'pct_it_workers_2018', 'median_hh_inc_2018']
col_set_2019 = ['cfips', 'pct_bb_2019', 'pct_college_2019', 'pct_foreign_born_2019', 'pct_it_workers_2019', 'median_hh_inc_2019']
col_set_2020 = ['cfips', 'pct_bb_2020', 'pct_college_2020', 'pct_foreign_born_2020', 'pct_it_workers_2020', 'median_hh_inc_2020']
col_set_2021 = ['cfips', 'pct_bb_2021', 'pct_college_2021', 'pct_foreign_born_2021', 'pct_it_workers_2021', 'median_hh_inc_2021']


sns.pairplot(data = census_data.loc[:,col_set_2021])



# Notes:

PCT BB is coorelated with past year. Easy to forecast with growth variable

PCR college may be coorelation to median hh income and pct bb
PCT BB and PCT income may have some degree of coorelation



In [ ]:
# thin_census_data
census_data

# Notes for later

1. No 2022 census data. May need to project to get estimates

# RE:Pre-Processing: After validation

Based on observations made, 
1. Census data lacks information for 2022, and needs to be generated/fetched from government website
2. Training data is not lacking any essential information

Steps:
1. Project/impute the 2022 census data into the dataset
2. Join the training set and the census data

#### Projecting census data

We can potentially use a slew of techniques here

### Merge the inputs provided

We will join the 2 datasets together on "cfips" and "year" columns.

In [ ]:
train_base

In [ ]:
train_data = train_base.merge(thin_census_data, on = ["cfips","year"], how = 'left')

# Using merge statament to avoid juggling index of dataframes
# We will validate the row count, which should not increase if join is correct
train_data = pd.merge(left = train_base,right = thin_census_data,how = 'left',on = ['cfips','year'])
train_data